In [ ]:
%pip install -q "openai>=1.40.0" "httpx>=0.27.2" "httpcore>=1.0.5" "pandas>=2.2.2" "tqdm>=4.66.4"

In [ ]:
import os, getpass, json, time, platform
from importlib.metadata import version, PackageNotFoundError

def ver(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return None

# ==== Configure DeepSeek ====
MODEL    = os.getenv("DEEPSEEK_MODEL", "Deepseek-V3")
BASE_URL = os.getenv("DEEPSEEK_BASE_URL", "https://api.deepseek.com")

api_key = os.getenv("DEEPSEEK_API_KEY") or os.getenv("API_KEY")
if not api_key:
    api_key = getpass.getpass("Paste your DeepSeek API key: ")
os.environ["DEEPSEEK_API_KEY"] = api_key

save_dir  = "./results_DeepseekV3"
meta_path = os.path.join(save_dir, "meta.json")

meta = {
    "model": MODEL,
    "base_url": BASE_URL,
    "python": platform.python_version(),
    "time_utc": time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),
    "openai_pkg": ver("openai"),
    "httpx": ver("httpx"),
}

os.makedirs(save_dir, exist_ok=True)
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Meta saved ->", meta_path)
print("MODEL:", MODEL, "| BASE_URL:", BASE_URL)
print("DEEPSEEK_API_KEY set:", bool(os.environ.get("DEEPSEEK_API_KEY")))


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"], base_url=BASE_URL)
print("Client ready.")

In [ ]:
# Text-only sanity check
messages = [
    {"role": "system", "content": "You are a concise assistant."},
    {"role": "user", "content": "Name two differences between breadth-first search and depth-first search."},
]
resp = client.chat.completions.create(model=MODEL, messages=messages, temperature=0.2, max_tokens=200)
print(resp.choices[0].message.content)

In [ ]:
# Tiny batch evaluation (toy): classify short lines as 'algo' vs 'not-algo'
import pandas as pd
from tqdm import tqdm
lines = [
    "Use Dijkstra with a binary heap to get O((V+E) log V).",
    "I microwaved my lunch today.",
    "We need to topologically sort the DAG before DP.",
]
def tagger(text):
    prompt = f"Output only 'algo' or 'not-algo' for: {text}"
    msgs = [
        {"role": "system", "content": "You are a terse classifier."},
        {"role": "user", "content": prompt},
    ]
    out = client.chat.completions.create(model=MODEL, messages=msgs, temperature=0.0, max_tokens=3)
    return out.choices[0].message.content.strip().lower()

rows = []
for t in tqdm(lines, desc="Scoring"):
    try:
        rows.append({"text": t, "prediction": tagger(t)})
    except Exception as e:
        rows.append({"text": t, "prediction": f"ERROR: {e!r}"})
df = pd.DataFrame(rows)
csv_path = os.path.join(save_dir, "toy_tags.csv")
df.to_csv(csv_path, index=False)
print("Saved:", csv_path)
df